In [2]:
%load_ext autoreload
%autoreload 2
import sys 
import plotly.express as px
import plotly.graph_objects as go
import re
import time

from lib import utils
from lib.slurm import SlurmClient, SlurmPolice

sp = SlurmPolice()
sl = SlurmClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Slurm Queue

In [2]:
q = sl.get_info()
q.NODES = q.NODES.astype(int)
print(q)
print("Total:", q.NODES.sum())

   STATE  NODES                                           NODELIST
0    mix     34  cpu-c7i12xlarge-1,h100-st-p548xlarge-[8-10,45-...
1   idle      1                                  cpu-c7i12xlarge-0
2   drng      1                               h100-st-p548xlarge-0
3  alloc    411  h100-st-p548xlarge-[1-7,11-44,47-61,65-66,68-7...
Total: 447


In [3]:

q = sl.get_queue_summary()

print("Total:", q.NODES.sum())
print("Total Running: ", q[q.ST=='R'].NODES.sum())
q

Total: 429
Total Running:  407


,ACCOUNT,ST,NODES
0,ar-ai-hipri,PD,6
1,ar-ai-hipri,R,260
2,ar-ai-voice-hipri,R,32
3,midpri,PD,16
4,midpri,R,115


In [20]:
q = sl.get_queue()
q.NAME.unique()
# q

array(['bash', 'eval_mmmu', 'ckpt_3920_600s',
       'pretrain_stage_2_perceiver_V3_70b_stage_2', 'interactive',
       'prod_mh19', 'prod_31', 'AGG_CAPTION_20240926_153554',
       'eval_ai2d_2000', 'eval_ai2d_1900', 'eval_ai2d_1800',
       'eval_ai2d_1700', 'eval_ai2d_1600', 'eval_ai2d_1500',
       'eval_ai2d_1400', 'eval_ai2d_1300', 'eval_ai2d_1200',
       'eval_ai2d_1100', 'eval_ai2d_1000', 'eval_infographics_1300',
       'eval_infographics_w_ocr_1400', 'eval_infographics_w_ocr_1500',
       'eval_infographics_w_ocr_1300', 'eval_infographics_w_ocr_1100',
       'eval_infographics_w_ocr_1200', 'eval_infographics_w_ocr_2000',
       'eval_infographics_w_ocr_1900', 'eval_infographics_w_ocr_1800',
       'eval_infographics_w_ocr_1600', 'eval_infographics_w_ocr_1700',
       'eval_infographics_w_ocr_1000', 'eval_infographics_1900',
       'eval_infographics_1800', 'eval_infographics_1700',
       'eval_infographics_1500', 'eval_infographics_1400',
       'eval_infographics_1000', 'e

In [7]:
q_ar.NODES.sum()

448

In [6]:
q_ar = sl.get_queue()
# q_ar = q_ar[(q_ar.ACCOUNT == 'ar-ai-hipri') & (q_ar.ST == 'R')]
q_ar = q_ar[(q_ar.ST == 'R')]
# q_ar = q_ar.group_by(['USER']).
fig = px.pie(q_ar, values='NODES', names='USER')
fig.update_traces(textinfo='label+value')
fig.show()
print(q_ar[['JOBID', 'ST', 'ACCOUNT', 'USER', 'NODES']])

    JOBID ST            ACCOUNT           USER  NODES
0   83508  R        ar-ai-hipri        cyprien      1
47  85060  R  ar-ai-voice-hipri   patrickhuber      1
48  83577  R  ar-ai-voice-hipri      zhaojiang     32
49  85029  R  ar-ai-voice-hipri      zhaojiang      1
50  85045  R             midpri         kapilk      1
51  85010  R             midpri         kapilk      1
52  85061  R             midpri         kapilk      1
53  85047  R             midpri         kapilk      1
54  85044  R             midpri         kapilk      1
55  85028  R             midpri         kapilk      1
56  85016  R             midpri         kapilk      1
57  85008  R             midpri         kapilk      1
58  85014  R             midpri       tomfujii      1
59  84995  R             midpri          tranx      1
60  84994  R             midpri          tranx      1
61  84993  R             midpri          tranx      1
62  84991  R             midpri          tranx      1
63  84992  R             mid

# Evict

In [8]:
sl.get_job_launch_cmd(97822)

cd fsx_0/user/cyprien/aligner_git/mmai_main/llm_mm_aligner/experiments/aws
sbatch --parsable /fsx_0/checkpoints/cyprien/mm9.2/MM9.2_SFT/MM9p2_LLama3.1_70B_ViTH_336px_exp28_plus_v2_1400_pdfa_qa_3m_only/241125_13_53_29_125877/launch_script.sh


In [12]:
sl = SlurmClient()
q = sl.get_queue()
# q[q.TIME < '5:00']
q

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_S
0,83508,ar-ai-hipri,jupyter,cyprien,R,2024-11-08T22:42:33,2-13:28:14,1,221294
1,84279,ar-ai-hipri,VEv0p2_G14_aws_final_128x8x98_next,marcomonteiro,PD,N/A,0:00,128,0
2,84904,midpri,eval_docvqa_19000,tranx,PD,2024-11-11T12:12:24,0:00,1,0
3,84906,midpri,eval_infographics_19000,tranx,PD,2024-11-11T12:12:24,0:00,1,0
4,84908,midpri,eval_ai2d_19000,tranx,PD,2024-11-11T12:12:24,0:00,1,0
5,84921,midpri,eval_chartqa_19200,tranx,PD,2024-11-11T12:12:24,0:00,1,0
6,84922,midpri,eval_docvqa_19200,tranx,PD,2024-11-11T12:12:24,0:00,1,0
7,84924,midpri,eval_infographics_19200,tranx,PD,2024-11-11T12:12:24,0:00,1,0
8,84902,midpri,eval_vqa_19000,tranx,PD,2024-11-11T12:12:15,0:00,1,0
9,84903,midpri,eval_chartqa_19000,tranx,PD,2024-11-11T12:12:15,0:00,1,0


In [15]:
sp = SlurmPolice()
sp.cancel(
    # users='tomfujii', 
    # name='eval_',
    # status=['PD'],
    # action='hold',
    action='release',
    # action='cancel',
    # name='eval_inf',
    # run_seconds = 600,
    # max_jobs=20
)

    JOBID  ST            ACCOUNT           USER  NODES
0   95413   R             midpri       ahmadyan      1
1   95100   R        ar-ai-hipri       tomfujii      1
2   97862   R        ar-ai-hipri        cyprien      1
3   97970  PD        ar-ai-hipri  marcomonteiro    256
4   97969  PD        ar-ai-hipri  marcomonteiro    256
5   97968  PD        ar-ai-hipri  marcomonteiro    256
6   97346  PD        ar-ai-hipri       ahmadyan     64
7   97972  PD             midpri        cyprien      1
8   97971  PD             midpri        cyprien      1
9   97965  PD             midpri      kgschwind      8
10  97964  PD             midpri      kgschwind      8
11  97952  PD             midpri        cyprien      1
12  97950  PD             midpri        cyprien      1
13  97948  PD             midpri        cyprien      1
14  97947  PD             midpri        cyprien      1
15  97946  PD             midpri        cyprien      1
16  97936  PD             midpri        parisah      1
17  97935 

In [8]:
sp.cancel(
    users='tranx', 
    # name='eval_dpo'
    status=['PD'],
    action='scancel'
)

    JOBID  ST ACCOUNT   USER  NODES
3   71205  PD  midpri  tranx      1
4   71204  PD  midpri  tranx      1
5   71206  PD  midpri  tranx      1
6   71207  PD  midpri  tranx      1
7   71208  PD  midpri  tranx      1
..    ...  ..     ...    ...    ...
70  71277  PD  midpri  tranx      1
71  71278  PD  midpri  tranx      1
72  71279  PD  midpri  tranx      1
73  71280  PD  midpri  tranx      1
74  71281  PD  midpri  tranx      1

[72 rows x 5 columns]
total nodes: 72
Triggering scancel on job 71205
Triggering scancel on job 71204
Triggering scancel on job 71206
Triggering scancel on job 71207
Triggering scancel on job 71208
Triggering scancel on job 71209
Triggering scancel on job 71210
Triggering scancel on job 71211
Triggering scancel on job 71212
Triggering scancel on job 71213
Triggering scancel on job 71214
Triggering scancel on job 71215
Triggering scancel on job 71216
Triggering scancel on job 71217
Triggering scancel on job 71218
Triggering scancel on job 71219
Triggering scance

In [4]:
sp.cancel(account='midpri', users=["yastashonok", "billwzhu"])

    JOBID ST ACCOUNT         USER  NODES
8   37132  R  midpri  yastashonok      2
9   37131  R  midpri  yastashonok      2
10  37130  R  midpri  yastashonok      2
total nodes: 6


Cancelling job 37132
Cancelling job 37131
Cancelling job 37130


In [75]:
# sp.cancel(account='midpri', users=['yfzhuang', 'jiuhai', 'imzyc'], test=False)

In [76]:
# sp.cancel(account='*', users=["yetian12"], test=False)

In [77]:
# sp.cancel(account='midpri', users=["prakash14"], test=False)

In [24]:
# evic interns
# while True:
interns = ['yfzhuang', 'jiuhai', 'imzyc', 'billwzhu']
sl = SlurmClient()
q = sl.get_queue()

q = q[q.USER.isin(interns)]
q = q[~q.ACCOUNT.isin(['ar-ai-research-interns', 'ar-ai-voice-hipri'])]

print(q[['JOBID', 'ST', 'ACCOUNT', 'USER', 'NODES']])

for j in list(q.JOBID):
    print(f"Cancelling job {j}")
    utils.get_bash_output(f"scancel {j}")

    # time.sleep(2)

   JOBID ST ACCOUNT      USER  NODES
2  36010  R  midpri  billwzhu      1
Cancelling job 36010


In [45]:
# # cancel all midpri
# q = sl.get_queue()
# q = q[(q.ACCOUNT=='midpri') & (q.ST=='R')]

# print(q[['JOBID', 'ST', 'ACCOUNT', 'USER', 'NODES']])

# for j in list(q.JOBID):
#     print(f"Cancelling job {j}")
#     utils.get_bash_output(f"scancel {j}")
    

Empty DataFrame
Columns: [JOBID, ST, ACCOUNT, USER, NODES]
Index: []


    JOBID  ST ACCOUNT      USER  NODES
9   32809  CG  midpri  yfzhuang      1
10  32799  CG  midpri  yfzhuang      1
Cancelling job 32809
Cancelling job 32799


In [17]:
q = sl.get_queue()
q = q[q.ACCOUNT.isin(['ar-ai-midpri', 'ar-ai-lowpri', 'all', 'midpri'])]

print(q[['JOBID', 'ST', 'ACCOUNT', 'USER', 'NODES']])
for j in list(q.JOBID):
    print(f"Cancelling job {j}")
    utils.get_bash_output(f"scancel {j}")

   JOBID ST ACCOUNT      USER  NODES
4  31408  R  midpri  tomfujii      1
5  31407  R  midpri  tomfujii      1
Cancelling job 31408
Cancelling job 31407


In [73]:
# evic me
q = sl.get_recent_queue('ar-ai-hipri')
q = q[q.USER == 'tranx']
# q = q[q.ST == 'RH']
q
# for _, job in q.iterrows():
# for j in q.JOBID:
#     if j != '12811':
#         utils.get_bash_output(f"scancel {j}")
#         print(f"Cancel {j}")

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
42,14330,ar-ai-hipri,s2_i18n,tranx,R,2024-08-22T17:49:45,1-00:54:45,64,89685
41,14332,ar-ai-hipri,s2_exp28,tranx,R,2024-08-22T17:23:50,1-01:20:40,128,91240
43,14171,ar-ai-hipri,s2_504,tranx,R,2024-08-22T00:59:10,1-17:45:20,64,150320
44,13932,ar-ai-hipri,bash,tranx,R,2024-08-21T19:21:55,1-23:22:35,1,170555


In [54]:
q = sl.get_queue_summary()
running = {}
for _, row in q.iterrows():
    if row.ST == 'R':
        running[row.ACCOUNT] = row.NODES
print(running)

quota = {'ar-ai-research-interns': 8,
         'ar-ai-voice-hipri': 50, 'ar-ai-hipri': 400}

evict_ids = {}
for acc in running:
    if acc not in quota:  # quota = 0
        print(acc)

{'all': 17, 'ar-ai-hipri': 274, 'ar-ai-midpri': 2, 'ar-ai-research-interns': 3, 'ar-ai-voice-hipri': 53}
all
ar-ai-midpri


In [30]:
q_summary = q.groupby(['ACCOUNT', 'ST']).aggregate(
    {'NODES': 'sum'}).reset_index()
q_summary

,ACCOUNT,ST,NODES
0,all,R,17
1,ar-ai-hipri,R,280
2,ar-ai-midpri,R,2
3,ar-ai-research-interns,R,3
4,ar-ai-voice-hipri,PD,94
5,ar-ai-voice-hipri,R,53


# Pretrain

In [48]:
MM9_CONF_DIR = "/fsx_0/user/tranx/experiments/llm_mm_aligner/stage1_mm9"
envvar = {
    "NUM": 7,
    "JSON_CONFIG": f"fbl_pretrain_MM9_70B_Llama31_336px_2nodes.json",
    # "CONDA_ENV": "aligner_v7",
}

sbatch_base = "/fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh"
sbatch_overwrite = {
    "job-name": "test",
    "nodes": 2,
    "ntasks": 2
}

MM9_CONF_DIR = "/fsx_0/user/tranx/experiments/llm_mm_aligner/stage1_mm9"
JSON_CONFIG = f"{MM9_CONF_DIR}/xxxfbl_pretrain_MM9_70B_Llama31_336px_2nodes.json"
ALIGNER_PARENT_DIR = "/fsx_0/user/tranx"

envvar_string = f"{JSON_CONFIG} {ALIGNER_PARENT_DIR}"

sbatch_vars_string = []
for k, v in sbatch_overwrite.items():
    sbatch_vars_string.append(f"--{k}={v}")
sbatch_vars_string = ' '.join(sbatch_vars_string)

# cmd = f'sbatch --parsable {sbatch_vars_string} {sbatch_base} {envvar_string}'
cmd = f'sbatch --parsable {sbatch_vars_string} {sbatch_base} a b'
print(cmd)

job_id = utils.get_bash_output(cmd, print_output=True)
job_id = int(job_id)

sbatch --parsable --job-name=test --nodes=2 --ntasks=2 /fsx_0/user/tranx/experiments/llm_mm_aligner/base_pretrain_MM9_70B_sbatch.sh a b
9659



In [29]:
!tail -f /fsx_0/user/tranx/output/slurm_logs/output_9402.txt

/var/spool/slurmd/job09402/slurm_script: line 23: conda: command not found
/var/spool/slurmd/job09402/slurm_script: line 24: conda: command not found
Using conda environment:
Error: CONDA_DEFAULT_ENV is not set to aligner_v7
^C


# Test trains

# Stage 2